In [9]:
import xml.etree.ElementTree as etree
import pandas as pd
import joblib
import os
import numpy as np
import matplotlib.pyplot as plt
import joblib
import math
import scipy.stats as stats

In [8]:
# use bolus insulin on board to fill in the bolus data
def iobCalcExponential(insulin, t, dia, peak):
    peak = 50
    end = dia * 60
    activityContrib = 0
    minsAgo = t
    iobContrib = 0
    if minsAgo < end:
        tau = peak * (1 - peak / end) / (1 - 2 * peak / end) # time constant of exponential decay
        a = 2 * tau / end # rise time factor
        S = 1 / (1 - a + (1 + a) * math.exp(-end / tau)) # auxiliary scale factor

    activityContrib = insulin * (S / tau ** 2) * minsAgo * (1 - minsAgo / end) * math.exp(-minsAgo / tau)
    # iobContrib = insulin * (1 - S * (1 - a) * ((minsAgo ** 2 / (tau * end * (1 - a)) - minsAgo / tau - 1) * math.exp(-minsAgo / tau) + 1))
    #print('DIA:', dia, 'minsAgo:', minsAgo, 'end:', end, 'peak:', peak, 'tau:', tau, 'a:', a, 'S:', S, 'activityContrib:', activityContrib, 'iobContrib:', iobContrib)

    return activityContrib

# use carb absoption curv to fill in the carb data
def carbAbsobCurv(carbintake, t):
    if t >= 0 and t < 15:
        return carbintake * (0.05 + 1 / 3 * t)
    elif t >= 15 and t < 45:
        return carbintake * (0.05 + 5 * (45 - t) / 30)
    elif t >= 45 and t <= 240:
        return carbintake * 0.05
    else:
        return 0

In [10]:
# read train and test data from pkl files
# glucose: blood glucose level
# finger: finger stick blood glucose level
# basal: basal insulin rate
# hr: heart rate
# gsr: galvanic skin response
# carbs: number of carbs consumed
# temp_basal: temporary basal rate
# dose: bolus insulin dose


file_dir = '../2020data'
train_data_list = []
for f in os.listdir(file_dir):
    if 'train' not in f:
        continue
    train_data = joblib.load(os.path.join(file_dir, f))
    new_train_data = np.zeros((train_data.shape[0], 5))
    new_train_data[:, 0] = train_data['glucose']
    new_train_data[:, 1] = train_data['dose']
    new_train_data[:, 2] = train_data['carbs']
    new_train_data[:, 3] = train_data['gsr']
    new_train_data[:, 4] = train_data['basal']
    # replace nan with 0
    new_train_data = np.nan_to_num(new_train_data)
    processed_train_data = np.zeros((new_train_data.shape[0], 5))
    processed_train_data[:, 0] = new_train_data[:, 0]
    processed_train_data[:, 3] = new_train_data[:, 3]
    processed_train_data[:, 4] = new_train_data[:, 4]
    # fill in bolus data (dose)
    new_bolus_iob = []
    for i in range(len(new_train_data)):
        if new_train_data[i][1] != 0:
            bolus_array = np.zeros((len(new_train_data)))
            for j in range(i, min(i + int(240 / 5) - 1, len(new_train_data))):
                bolus_array[j] = iobCalcExponential(new_train_data[i][1], (j - i) * 5, 4, 50)
            new_bolus_iob.append(bolus_array)
    # add bolus data into one array
    new_bolus_iob = np.array(new_bolus_iob)
    new_bolus_iob = np.sum(new_bolus_iob, axis=0)
    processed_train_data[:, 1] = new_bolus_iob
    # fill in carb data (carbs)
    new_carb_absob = []
    for i in range(len(new_train_data)):
        if new_train_data[i][2] != 0:
            carb_array = np.zeros((len(new_train_data)))
            for j in range(i, min(i + int(240 / 5) - 1, len(new_train_data))):
                carb_array[j] = carbAbsobCurv(new_train_data[i][2], (j - i) * 5)
            new_carb_absob.append(carb_array)
    # add carb data into one array
    new_carb_absob = np.array(new_carb_absob)
    new_carb_absob = np.sum(new_carb_absob, axis=0)
    processed_train_data[:, 2] = new_carb_absob
    train_data_list.append(processed_train_data)

test_data_list = []
for f in os.listdir(file_dir):
    if 'test' not in f:
        continue
    test_data = joblib.load(os.path.join(file_dir, f))
    new_test_data = np.zeros((test_data.shape[0], 5))
    new_test_data[:, 0] = test_data['glucose']
    new_test_data[:, 1] = test_data['dose']
    new_test_data[:, 2] = test_data['carbs']
    new_test_data[:, 3] = test_data['gsr']
    new_test_data[:, 4] = test_data['basal']
    # replace nan with 0
    new_test_data = np.nan_to_num(new_test_data)
    processed_test_data = np.zeros((new_test_data.shape[0], 5))
    processed_test_data[:, 0] = new_test_data[:, 0]
    processed_test_data[:, 3] = new_test_data[:, 3]
    processed_test_data[:, 4] = new_test_data[:, 4]
    # fill in bolus data (dose)
    new_bolus_iob = []
    for i in range(len(new_test_data)):
        if new_test_data[i][1] != 0:
            bolus_array = np.zeros((len(new_test_data)))
            for j in range(i, min(i + int(240 / 5) - 1, len(new_test_data))):
                bolus_array[j] = iobCalcExponential(new_test_data[i][1], (j - i) * 5, 4, 50)
            new_bolus_iob.append(bolus_array)
    # add bolus data into one array
    new_bolus_iob = np.array(new_bolus_iob)
    new_bolus_iob = np.sum(new_bolus_iob, axis=0)
    processed_test_data[:, 1] = new_bolus_iob
    # fill in carb data (carbs)
    new_carb_absob = []
    for i in range(len(new_test_data)):
        if new_test_data[i][2] != 0:
            carb_array = np.zeros((len(new_test_data)))
            for j in range(i, min(i + int(240 / 5) - 1, len(new_test_data))):
                carb_array[j] = carbAbsobCurv(new_test_data[i][2], (j - i) * 5)
            new_carb_absob.append(carb_array)
    # add carb data into one array
    new_carb_absob = np.array(new_carb_absob)
    new_carb_absob = np.sum(new_carb_absob, axis=0)
    processed_test_data[:, 2] = new_carb_absob
    test_data_list.append(processed_test_data)

In [11]:
print('train_data_list length:', len(train_data_list))
print('test_data_list length:', len(test_data_list))

train_data_list length: 12
test_data_list length: 12


In [12]:
# split the data based on given backcast_length and forecast_length
backcast_length = 24
forecast_length = 6

final_train_df_list = []
final_test_df_list = []

for train_data in train_data_list:
    train_df_list = []
    for i in range(len(train_data) - backcast_length - forecast_length + 1):
        train_data_sample = train_data[i:i + backcast_length + forecast_length]
        # check if there is zero in the glucose data
        if 0 in train_data_sample[:, 0]:
            continue
        final_train_df_list.append(train_data[i:i + backcast_length + forecast_length])
for test_data in test_data_list:
    test_df_list = []
    for i in range(len(test_data) - backcast_length - forecast_length + 1):
        test_data_sample = test_data[i:i + backcast_length + forecast_length]
        # check if there is zero in the glucose data
        if 0 in test_data_sample[:, 0]:
            continue
        final_test_df_list.append(test_data[i:i + backcast_length + forecast_length])

final_train_df_list = np.array(final_train_df_list)
final_test_df_list = np.array(final_test_df_list)

# split train into train and val
train_df_list = final_train_df_list[:int(len(final_train_df_list) * 0.8)]
val_df_list = final_train_df_list[int(len(final_train_df_list) * 0.8):]
test_df_list = final_test_df_list

In [13]:
print('train_df_list length:', len(train_df_list))
print('val_df_list length:', len(val_df_list))
print('test_df_list length:', len(test_df_list))

train_df_list length: 97832
val_df_list length: 24458
test_df_list length: 28656


In [14]:
# save train, val, test data in npy format
np.save('./ohio_30/train.npy', train_df_list)
np.save('./ohio_30/val.npy', val_df_list)
np.save('./ohio_30/test.npy', test_df_list)

In [15]:
# split the data based on given backcast_length and forecast_length
backcast_length = 48
forecast_length = 12

final_train_df_list = []
final_test_df_list = []

for train_data in train_data_list:
    train_df_list = []
    for i in range(len(train_data) - backcast_length - forecast_length + 1):
        train_data_sample = train_data[i:i + backcast_length + forecast_length]
        # check if there is zero in the glucose data
        if 0 in train_data_sample[:, 0]:
            continue
        final_train_df_list.append(train_data[i:i + backcast_length + forecast_length])
for test_data in test_data_list:
    test_df_list = []
    for i in range(len(test_data) - backcast_length - forecast_length + 1):
        test_data_sample = test_data[i:i + backcast_length + forecast_length]
        # check if there is zero in the glucose data
        if 0 in test_data_sample[:, 0]:
            continue
        final_test_df_list.append(test_data[i:i + backcast_length + forecast_length])

final_train_df_list = np.array(final_train_df_list)
final_test_df_list = np.array(final_test_df_list)

# split train into train and val
train_df_list = final_train_df_list[:int(len(final_train_df_list) * 0.8)]
val_df_list = final_train_df_list[int(len(final_train_df_list) * 0.8):]
test_df_list = final_test_df_list

In [16]:
print('train_df_list length:', len(train_df_list))
print('val_df_list length:', len(val_df_list))
print('test_df_list length:', len(test_df_list))

train_df_list length: 87988
val_df_list length: 21997
test_df_list length: 25549


In [17]:
# save train, val, test data in npy format
np.save('./ohio_60/train.npy', train_df_list)
np.save('./ohio_60/val.npy', val_df_list)
np.save('./ohio_60/test.npy', test_df_list)